在Ocaml中，`let`绑定是不可变的。尽管Ocaml有很多可变的值，不过不存在可变的变量。

In [2]:
open Core

In [3]:
let (ints, strings) = List.unzip [(1, "one"); (2, "two"); (3, "three")];;

val ints : int Core.List.t = [1; 2; 3]
val strings : string Core.List.t = ["one"; "two"; "three"]


这里的(ints, strings)是一个模式，`let`绑定会为出现在该模式的两个标识符赋值。模式实际上就是对**数据结构形状的一个描述**，其中一些分量是要绑定的标识符。

 在`let`绑定中使用模式对于”不可否定“的模式最有意义，也就是说，当前类型中的所有值都肯定能与该模式匹配。元祖和记录模式就是不可否定的，但列表模式并非如此。

In [4]:
let upcase_first_entry line = let (first::rest) = String.split ~on:',' line in 
String.concat ~sep:"," (String.uppercase first::rest)

File "[4]", line 1, characters 34-47:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
[]


val upcase_first_entry : Core.String.t -> Core.String.t = <fun>


实际中这种情况并不会出现，因为`String.split`总会返回一个至少包含一个元素的列表。不过，编译器并不知道这一点，所以它还是会发出警告。通常最好使用一个`match`语句来显示处理这些情况。

In [7]:
let upcase_first_entry line = 
match String.split ~on:',' line with
| [] -> assert false
| first :: rest -> String.concat ~sep:"," (String.uppercase first::rest)

val upcase_first_entry : Core.String.t -> Core.String.t = <fun>


可以应用一个科里函数来得到一个新函数，这种做法称为部分应用。

In [10]:
let rec find_first_shutter list = 
match list with
|[] | [_] ->
None
| x :: y :: tl ->
if x = y then Some x else find_first_shutter(y::tl)

val find_first_shutter : 'a list -> 'a option = <fun>


In [13]:
find_first_shutter [2;4;5;5;2;1]

- : int option = Some 5


Ocaml区分了非递归定义（使用`let`）和递归定义（使用`let rec`），这很大程度上是由于技术上的原因：如果一组函数定义是相互递归的，类型推断算法必须要知道这一点。

如果一个中缀操作符两边加上括号，就可以把它当作一个常规的前缀函数使用了

In [14]:
(+) 3 4

- : int = 7


In [15]:
let (+!) (x1, y1) (x2, y2) = (x1 + x2, y1 + y2)

val ( +! ) : int * int -> int * int -> int * int = <fun>


In [23]:
(3,2) +! (-2,4)

- : int * int = (1, 6)


`|>`还扮演一个很有用的角色，能够对操作符排序，从本质上类似于`Unix shell`中使用的管道字符。例如，考虑下面的代码，它会打印出`PATH`的各个不同的元素。需要说明，后面的`List.dedup`会删除列表中重复的元素，并将使用所提供的比较函数对列表排序

In [19]:
let path = "/usr/bin:/usr/local/bin:/bin:/sbin"

val path : string = "/usr/bin:/usr/local/bin:/bin:/sbin"


In [22]:
String.split ~on:':' path
|> List.dedup ~compare:String.compare
|> List.iter ~f:print_endline

File "[22]", line 2, characters 3-13:
Warning 3: deprecated: Core.List.dedup
[since 2017-04] Use [dedup_and_sort] instead


/bin
/sbin
/usr/bin
/usr/local/bin


- : unit = ()


In [25]:
要注意，不用`|>`也可以做到这一点，不过结果有点繁琐：

File "[25]", line 1, characters 0-1:
Warning 3: deprecated: ISO-Latin1 characters in identifiers


In [26]:
let split_path = String.split ~on：':' path in
let deduped_path  = List.dedup ~compare:String.compare split_path in
List.iter ~f:print_endline deduped_path

File "[26]", line 1, characters 31-34:
Warning 3: deprecated: ISO-Latin1 characters in identifiers


`(^>)`是右结合的

In [29]:
let (^>) x  f = f x;;

val ( ^> ) : 'a -> ('a -> 'b) -> 'b = <fun>


In [30]:
Sys.getenv_exn "PATH"
^> String.split ~on:':' path
^> List.dedup ~compare:String.compare
^> List.iter ~f:print_endline

File "[30]", line 3, characters 3-13:
Warning 3: deprecated: Core.List.dedup
[since 2017-04] Use [dedup_and_sort] instead


error: compile_error

这里的问题是，由于`^>`是右结合的，这个操作符试图把值`List.dup ~compare:String.compare`传入函数`List.iter` `~f:print_endline`。不过`List.iter~f:print_endline`希望得到一个字符串列表作为输入，而不是一个函数。

In [31]:
let some_or_zero = function
| Some x -> x
| None -> 0

val some_or_zero : int option -> int = <fun>


In [33]:
List.map ~f:some_or_zero [Some 3; None; Some 4];;

- : int Core.List.t = [3; 0; 4]


这等价于将一个常规的函数定义与一个`match`相结合

In [34]:
let some_or_zero num_opt = 
match num_opt with
|Some x -> x
|None -> 0


val some_or_zero : int option -> int = <fun>


还可以把不同的函数声明结合在一起，如下面的例子所示，这里我们声明了一个链参数（科里）函数，并对第二个参数进完成了一个模式匹配

In [37]:
let some_or_default default = function
| Some x -> x
| None -> default

val some_or_default : 'a -> 'a option -> 'a = <fun>


In [40]:
some_or_default 3 (Some 5);;

- : int = 5


In [42]:
List.map ~f:(some_or_default 100) [Some 3; None; Some 4];;

- : int Core.List.t = [3; 100; 4]


In [46]:
let ratio ~num ~denom = float num /. float denom;;

val ratio : num:int -> denom:int -> float = <fun>


标签参数由前面的一个波浪线标志，并把标签（后面有一个冒号）放在要加标签的变量前面（实际上这里使用了标签双关，所以省略了冒号和后面的同名变量）。

可以按任意的顺序提供参数

In [48]:
ratio ~num:3 ~denom:10;;
ratio ~denom:10 ~num:3;;

- : float = 0.3


- : float = 0.3


In [50]:
let num = 3 in
let denom = 4 in
ratio ~num ~denom

- : float = 0.75


In [51]:
let apply_to_tuple f (first, second) = f ~first ~second;

val apply_to_tuple : (first:'a -> second:'b -> 'c) -> 'a * 'b -> 'c = <fun>


第二种方式改变了便签参数列出的顺序：

In [52]:
let apply_to_tuple_2 f (first, second) = f ~second ~first;;

val apply_to_tuple_2 : (second:'a -> first:'b -> 'c) -> 'b * 'a -> 'c = <fun>


In [53]:
let divide ~first ~second = first / second;;

val divide : first:int -> second:int -> int = <fun>


In [54]:
apply_to_tuple_2 divide (3, 4);;

error: compile_error

In [56]:
apply_to_tuple divide (3, 4);;

- : int = 0


因此，将标签函数作为参数传入时，需要保证标签参数顺序一致

In [58]:
let concat ?sep x y = 
let sep = match sep with None -> "" | Some x -> x in
x ^ sep ^ y

val concat : ?sep:string -> string -> string -> string = <fun>


In [59]:
concat "foo" "bar"

- : string = "foobar"


In [61]:
concat ~sep:":" "foo" "bar"

- : string = "foo:bar"


前面的例子需要某种“样板”，如果什么也没有提供，要选择一个默认的风格符号。这是一个更简洁的`concat`

In [62]:
let concat ?(sep="") x y = x ^ sep ^ y;;

val concat : ?sep:string -> string -> string -> string = <fun>


下面的两行都指定了`concat`的sep参数，它们是等价的

In [64]:
concat ~sep:":" "foo" "bar"

- : string = "foo:bar"


In [66]:
concat ?sep:(Some ":") "foo" "bar"

- : string = "foo:bar"


下面两行代码调用`concat`时未指定`sep`，它们也是等价的

In [67]:
concat  "foo" "bar"

- : string = "foobar"


In [68]:
concat ?sep:None "foo" "bar"

- : string = "foobar"


在底层，对于一个有可选参数的函数，如果调用者没有提供这个参数，函数会接收`None`，如果调用者提供了这个参数，则会接受·Some·

由于看起来有多个类型可以选择，`Ocaml`选择时需要有一些启发式规则。**编译器使用的启发式规则是尽量使用标签而不是选项，另外要选择参数在源代码中出现的顺序**

In [69]:
let numeric_deriv ~delta ~x ~y ~(f: x:float -> y:float -> float) = 
let x' = x +. delta in
let y' = y +. delta in
let base = f ~x ~y in
let dx = (f ~y ~x:x' -. base) /. delta in
let dy = (f ~x ~y:y' -. base) /. delta in
(dx, dy)

val numeric_deriv :
  delta:float ->
  x:float -> y:float -> f:(x:float -> y:float -> float) -> float * float =
  <fun>


In [71]:
let numeric_deriv ~delta ~x ~y ~f = 
let x' = x +. delta in
let y' = y +. delta in
let base = f ~x ~y in
let dx = (f ~y ~x:x' -. base) /. delta in
let dy = (f ~x ~y:y' -. base) /. delta in
(dx, dy)

error: compile_error

正如错误信息所示，可以让Ocaml接受这一点，也就是说，如果我们提供了显式的类型信息，指出会议不同的参数顺序使用`f`，`Ocaml`就不会抱怨了。下面的代码为`f`增加了类型标注，因此可以顺利编译，不会有错误

In [72]:
let colon_concat = concat ~sep:":";;

val colon_concat : string -> string -> string = <fun>


In [73]:
colon_concat "a" "b";;

- : string = "a:b"


In [74]:
let prepend_pound = concat "# "

val prepend_pound : string -> string = <fun>


In [75]:
prepend_pound "a BASH comment"

- : string = "# a BASH comment"


可选参数`?sep`现在消失了，或者说被擦除了。是几扇个，如果现在试图传入这个可选参数，则会被拒绝

In [76]:
prepend_pound "a BASH comment" ~sep:":";;

error: compile_error

基本原则是：可选参数后面第一个位置上定义的参数（也就是说，不能是标签参数或可选参数）一旦传入，这个可选参数就会被擦除，这就解释了`prepend_pound`的行为。不过，如果定义`concat`时把可选参数放到第二个位置上

In [77]:
let concat x ?(sep="") y = x ^ sep ^ y;;

val concat : string -> ?sep:string -> string -> string = <fun>


In [78]:
let prepend_pound = concat "# "

val prepend_pound : ?sep:string -> string -> string = <fun>


In [79]:
prepend_pound "A BASH comment";;

- : string = "# A BASH comment"


In [80]:
prepend_pound "a BASH comment" ~sep:"--- "

- : string = "# --- a BASH comment"
